In [1]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
from numpy.random import rand

In [2]:
class BaseProblem:
    def __init__(self, initial_state):
        self.state = initial_state

    @staticmethod
    def energy(state):
        """
        Calculate the energy of the current state.
        This function should be overridden by subclasses to define a specific energy function.
        """
        raise NotImplementedError(
            "The energy function must be overridden by subclasses"
        )

    def neighbor(self):
        """
        Propose a random neighbor state.
        This function should be overridden by subclasses to define how to generate a neighbor state.
        """
        raise NotImplementedError(
            "The neighbor function must be overridden by subclasses"
        )

In [3]:
%matplotlib qt

In [4]:
class Sudoku(BaseProblem):
    def __init__(self, constraint):

        self.constraint = constraint
        self.dim = constraint.shape[0]
        assert np.all(self.constraint <= self.dim)
        super().__init__(self.random_state())

    def random_state(self):
        return np.where(
            self.constraint > 0,
            self.constraint,
            np.random.randint(1, self.dim, size=(self.dim, self.dim)),
        )

    @staticmethod
    def energy(state):
        repeats = 0
        # Check rows for repeats
        for row in state:
            repeats += len(row) - len(set(row))
        # Check columns for repeats
        for col in state.T:
            repeats += len(col) - len(set(col))

        # Check doubly-square subgrids for repeats
        dim = int(state.shape[0])
        subdim = np.sqrt(dim)
        if int(subdim) == subdim:
            subdim = int(subdim)
            for i in range(0, dim, subdim):
                for j in range(0, dim, subdim):
                    subgrid = state[i : i + subdim, j : j + subdim].flatten()
                    repeats += len(subgrid) - len(set(subgrid))

        return repeats

    def random_neighbor(self):
        new_state = self.state.copy()
        while True:
            i, j = np.random.randint(0, self.dim, size=2)
            if not self.constraint[i, j]:
                new_state[i, j] = np.random.randint(1, self.dim + 1)
                return new_state

    def update(self, new_state):
        self.state = new_state

    def copy(self):
        new_instance = Sudoku(self.constraint.copy())
        new_instance.state = self.state.copy()
        return new_instance


class Sudoku9(BaseProblem):
    def __init__(self, constraint):
        self.constraint = constraint
        self.dim = constraint.shape[0]
        assert self.dim == 9
        assert np.all(self.constraint <= self.dim)
        super().__init__(self.random_state())

    def random_state(self):
        state = np.zeros((self.dim, self.dim), dtype=int)
        subdim = int(np.sqrt(self.dim))
        for i in range(0, self.dim, subdim):
            for j in range(0, self.dim, subdim):
                loc_constr = self.constraint[i : i + subdim, j : j + subdim].flatten()
                do_not_use = set(loc_constr)
                use = list(set(range(1, self.dim + 1)) - do_not_use)
                np.random.shuffle(use)
                loc_constr[loc_constr == 0] = use[: np.sum(loc_constr == 0)]
                state[i : i + subdim, j : j + subdim] = loc_constr.reshape(
                    (subdim, subdim)
                )

        return state

    @staticmethod
    def energy(state):
        repeats = 0
        # Check rows for repeats
        for row in state:
            repeats += len(row) - len(set(row))

        # Check columns for repeats
        for col in state.T:
            repeats += len(col) - len(set(col))

        # Check doubly-square subgrids for repeats
        dim = int(state.shape[0])
        subdim = np.sqrt(dim)
        if int(subdim) == subdim:
            subdim = int(subdim)
            for i in range(0, dim, subdim):
                for j in range(0, dim, subdim):
                    subgrid = state[i : i + subdim, j : j + subdim].flatten()
                    repeats += len(subgrid) - len(set(subgrid))

        return repeats

    def random_neighbor(self):
        new_state = self.state.copy()
        subdim = int(np.sqrt(self.dim))

        while True:
            mu, nu, i0, j0, i1, j1 = np.random.randint(0, subdim, size=6)
            if not (
                self.constraint[i0 + mu * subdim, j0 + nu * subdim]
                or self.constraint[i1 + mu * subdim, j1 + nu * subdim]
            ):
                temp = new_state[i0 + mu * subdim, j0 + nu * subdim]
                new_state[i0 + mu * subdim, j0 + nu * subdim] = new_state[
                    i1 + mu * subdim, j1 + nu * subdim
                ]
                new_state[i1 + mu * subdim, j1 + nu * subdim] = temp
                return new_state

    def update(self, new_state):
        self.state = new_state

    def copy(self):
        new_instance = Sudoku9(self.constraint.copy())
        new_instance.state = self.state.copy()
        return new_instance

In [101]:
constraint = np.zeros((5, 5), dtype=int)
constraint[0, 0] = 1
constraint[1, 1] = 2
constraint[2, 2] = 3
constraint[3, 3] = 4
constraint[4, 4] = 5
test = Sudoku(constraint)

In [5]:
class Annealer:
    def __init__(
        self, problem, initial_temp, cooling_rate, min_temp, reheating_factor=10
    ):
        self.problem = problem
        self.T0 = initial_temp
        self.cooling_rate = cooling_rate
        self.min_temp = min_temp
        self.reheating_factor = reheating_factor

    def cool_down(self, T):
        return T * self.cooling_rate

    def reheat(self, T):
        return T * self.reheating_factor

    def accept(self, energy_diff, T):
        if energy_diff <= 0:
            return True
        else:
            return np.random.rand() < np.exp(-energy_diff / T)

    def anneal(self, steps_per_temp, max_cooling_steps):
        problem = self.problem.copy()
        current_state = problem.state
        current_energy = problem.energy(current_state)
        T = self.T0
        energy_history = np.zeros(max_cooling_steps)
        temp_history = np.zeros(max_cooling_steps)

        for ostep in range(max_cooling_steps):
            if T < self.min_temp:
                print(f"Temperature fell below minimum temperature ({T})")
                break

            for istep in range(steps_per_temp):
                new_state = problem.random_neighbor()
                new_energy = problem.energy(new_state)
                energy_diff = new_energy - current_energy

                if self.accept(energy_diff, T):
                    problem.update(new_state)
                    current_energy = new_energy
                    current_state = new_state
                    if current_energy == 0:
                        print(f"Found solution in {(ostep+1)*(istep+1)} cooling steps")
                        print(f"Final temperature: {T}")
                        return current_state, current_energy
            energy_history[ostep] = current_energy
            temp_history[ostep] = T

            window = 25
            if ostep % window == 0:
                plt.clf()
                plt.subplot(2, 1, 1)
                plt.plot(energy_history[:ostep], label="Energy")
                plt.ylabel("Energy")
                # plt.ylim([0, 1.35 * energy_history[0]])
                plt.legend()

                plt.subplot(2, 1, 2)
                plt.plot(temp_history[:ostep], label="Temperature")
                plt.yscale("log")
                plt.ylim([self.min_temp, 10 * self.T0])
                plt.xlabel("Cooling Steps")
                plt.ylabel("Temperature (log scale)")
                plt.legend()
                plt.pause(0.01)
                if ostep >= window and np.all(
                    energy_history[ostep - window : ostep] == energy_history[ostep - 1]
                ):
                    T = self.reheat(T)

            T = self.cool_down(T)

        print("Failed to cool to the ground state")
        return current_state, current_energy

In [48]:
annealer = Annealer(problem=test, initial_temp=5, cooling_rate=0.99, min_temp=0.01)

In [49]:
energy, state = annealer.anneal(steps_per_temp=25 * 2, max_cooling_steps=1000)

Found solution in 6994 cooling steps
Final temperature: 0.3382222361003227


In [50]:
print(energy)
print(state)

[[1 5 4 2 3]
 [5 2 1 3 4]
 [4 1 3 5 2]
 [2 3 5 4 1]
 [3 4 2 1 5]]
0


In [ ]:
annealer.problem.energy(annealer.problem.state)

In [6]:
nyt = np.zeros((9, 9), dtype=int)
nyt[0, 3] = 8
nyt[0, 4] = 6
nyt[1, 1] = 3
nyt[1, 5] = 1
nyt[1, 8] = 7
nyt[2, 3] = 4
nyt[2, 4] = 7
nyt[2, 7] = 9
nyt[3, 6] = 5
nyt[4, 0] = 2
nyt[4, 1] = 4
nyt[4, 2] = 9
nyt[4, 5] = 5
nyt[4, 7] = 3
nyt[5, 0] = 7
nyt[5, 6] = 4
nyt[6, 1] = 7
nyt[6, 5] = 2
nyt[6, 7] = 1
nyt[7, 0] = 8
nyt[7, 2] = 1
nyt[7, 3] = 9
nyt[7, 8] = 5
nyt[8, 8] = 6
print(nyt)

[[0 0 0 8 6 0 0 0 0]
 [0 3 0 0 0 1 0 0 7]
 [0 0 0 4 7 0 0 9 0]
 [0 0 0 0 0 0 5 0 0]
 [2 4 9 0 0 5 0 3 0]
 [7 0 0 0 0 0 4 0 0]
 [0 7 0 0 0 2 0 1 0]
 [8 0 1 9 0 0 0 0 5]
 [0 0 0 0 0 0 0 0 6]]


In [7]:
problem = Sudoku9(nyt)

In [8]:
annealer = Annealer(
    problem=problem,
    initial_temp=15,
    cooling_rate=0.995,
    min_temp=0.01,
    reheating_factor=2.5,
)

In [9]:
annealer.anneal(81 * 4, 3000)

Found solution in 59856 cooling steps
Final temperature: 0.2955880675066096


(array([[1, 2, 7, 8, 6, 9, 3, 5, 4],
        [9, 3, 4, 5, 2, 1, 8, 6, 7],
        [5, 8, 6, 4, 7, 3, 1, 9, 2],
        [6, 1, 3, 2, 4, 8, 5, 7, 9],
        [2, 4, 9, 7, 1, 5, 6, 3, 8],
        [7, 5, 8, 3, 9, 6, 4, 2, 1],
        [4, 7, 5, 6, 8, 2, 9, 1, 3],
        [8, 6, 1, 9, 3, 7, 2, 4, 5],
        [3, 9, 2, 1, 5, 4, 7, 8, 6]]),
 0)